In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_excel('/content/drive/MyDrive/NNFL-Assignment2/data5.xlsx', header= None)
df.head()

,0,1,2,3,4,5,6,7
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1


In [4]:
df = df.drop_duplicates(keep='first')
df = df.sample(frac = 1, random_state = 7).reset_index(drop=True)
m = df.shape[0]
n = df.shape[1] - 1
print(m,n)
df.head()

210 7


,0,1,2,3,4,5,6,7
0,18.94,16.49,0.8750,6.445,3.639,5.0640,6.362,2
1,12.88,13.50,0.8879,5.139,3.119,2.3520,4.607,1
2,18.36,16.52,0.8452,6.666,3.485,4.9330,6.448,2
3,15.88,14.90,0.8988,5.618,3.507,0.7651,5.091,1
4,13.22,13.84,0.8680,5.395,3.070,4.1570,5.088,1


In [5]:
datan = df.values
X = datan[:, 0:7]
y = datan[:, -1:]

num_classes = 3

#X = (X - np.min(X, axis = 0))/(np.max(X, axis=0)-np.min(X, axis=0))
X = (X - X.mean(axis = 0))/(X.std(axis = 0))
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1)
#print(X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape)
X_train = X_train.T
X_val = X_val.T
X_test = X_test.T


one_hot_y_train = np.zeros((y_train.shape[0], num_classes))
one_hot_y_test = np.zeros((y_test.shape[0], num_classes))
one_hot_y_val = np.zeros((y_val.shape[0], num_classes))

for i in range(y_train.shape[0]):
  one_hot_y_train[i, int(y_train[i])-1] = 1

for i in range(y_test.shape[0]):
  one_hot_y_test[i, int(y_test[i])-1] = 1

for i in range(y_val.shape[0]):
  one_hot_y_val[i, int(y_val[i])-1] = 1



one_hot_y_train = one_hot_y_train.T
one_hot_y_test = one_hot_y_test.T
one_hot_y_val = one_hot_y_val.T

print(X_train.shape,X_val.shape, X_test.shape, one_hot_y_train.shape, one_hot_y_test.shape, one_hot_y_val.shape)

(7, 151) (7, 17) (7, 42) (3, 151) (3, 42) (3, 17)


In [6]:
def relu(x):
  return np.maximum(0, x)


def softmax(x):
  expX = np.exp(x)
  return expX/np.sum(expX)

def sigmoid(x):
  return 1/(1+np.exp(-x))

def tanh(x):
  return np.tanh(x)

In [7]:
def relu_der(x):
  return np.array(x > 0, dtype=np.float32)

def sigmoid_der(x):
  sig = sigmoid(x)
  return sig * (1-sig)

def tanh_der(x):
  return (1 - np.power(np.tanh(x), 2))

In [8]:
def initialize_params(n_x, n_h1, n_h2, n_y):
  w1 = np.random.randn(n_h1, n_x)*0.01
  b1 = np.zeros((n_h1, 1))

  w2 = np.random.randn(n_h2, n_h1)*0.01
  b2 = np.zeros((n_h2, 1))

  w3 = np.random.randn(n_y, n_h2)*0.01
  b3 = np.zeros((n_y, 1))

  parameters = {
      "w1" : w1,
      "b1" : b1,
      "w2" : w2,
      "b2" : b2,
      "w3" : w3,
      "b3" : b3
  }

  return parameters

In [9]:
def forward_prop(x, parameters):
  w1 = parameters['w1']
  b1 = parameters['b1']
  w2 = parameters['w2']
  b2 = parameters['b2']
  w3 = parameters['w3']
  b3 = parameters['b3']

  z1 = np.dot(w1, x) + b1
  a1 = relu(z1)

  z2 = np.dot(w2, a1) + b2
  a2 = relu(z2)

  z3 = np.dot(w3, a2) + b3
  a3 = softmax(z3)

  cache = {
      'z1': z1,
      'a1': a1,
      'z2': z2,
      'a2': a2,
      'z3': z3,
      'a3': a3
  }

  return cache

In [10]:
def cost_function(a3, y):
  m = y.shape[1]
  cost = -(1/m)*np.sum(y*np.log(a3))#a3 -> 3xm, y -> 3xm
  return cost

In [11]:
def backward_prop(x, y, parameters, cache):
    
    w1 = parameters['w1']
    b1 = parameters['b1']
    w2 = parameters['w2']
    b2 = parameters['b2']
    w3 = parameters['w3']
    b3 = parameters['b3']

    a1 = cache['a1']
    a2 = cache['a2']
    a3 = cache['a3']
    
    m = x.shape[1]
    
    dz3 = (a3 - y)
    dw3 = (1/m)*np.dot(dz3, a2.T)
    db3 = (1/m)*np.sum(dz3, axis = 1, keepdims = True)


    dz2 = np.dot(w3.T, dz3)*relu_der(a2)
    dw2 = (1/m)*np.dot(dz2, a1.T)
    db2 = (1/m)*np.sum(dz2, axis = 1, keepdims = True)
    

    dz1 = np.dot(w2.T, dz2)*relu_der(a1)
    dw1 = (1/m)*np.dot(dz1, x.T)
    db1 = (1/m)*np.sum(dz1, axis = 1, keepdims = True)
    
    gradients = {
        "dw1" : dw1,
        "db1" : db1,
        "dw2" : dw2,
        "db2" : db2,
        "dw3" : dw3,
        "db3" : db3
    }
    
    return gradients



In [12]:
def update_parameters(parameters, gradients, learning_rate):
    
    w1 = parameters['w1']
    b1 = parameters['b1']
    w2 = parameters['w2']
    b2 = parameters['b2']
    w3 = parameters['w3']
    b3 = parameters['b3']
    
    dw1 = gradients['dw1']
    db1 = gradients['db1']
    dw2 = gradients['dw2']
    db2 = gradients['db2']
    dw3 = gradients['dw3']
    db3 = gradients['db3']

    w1 = w1 - learning_rate*dw1
    b1 = b1 - learning_rate*db1
    w2 = w2 - learning_rate*dw2
    b2 = b2 - learning_rate*db2
    w3 = w3 - learning_rate*dw3
    b3 = b3 - learning_rate*db3

    parameters = {
        "w1" : w1,
        "b1" : b1,
        "w2" : w2,
        "b2" : b2,
        "w3" : w3,
        "b3" : b3
    }
    
    return parameters



In [13]:
def model(x, y, n_h1, n_h2, learning_rate, iterations):
    
    n_x = x.shape[0]
    n_y = y.shape[0]
    
    cost_list = []
    
    parameters = initialize_params(n_x, n_h1 ,n_h2, n_y)
    
    for i in range(iterations):
        
        cache = forward_prop(x, parameters)
        
        cost = cost_function(cache['a3'], y)
        
        gradients = backward_prop(x, y, parameters, cache)
        
        parameters = update_parameters(parameters, gradients, learning_rate)
        
        cost_list.append(cost)
        
    return parameters, cost_list


In [14]:
N1 = [5, 10, 15, 30, 60, 120, 240, 480, 960, 1920]
N2 = [5, 10, 15, 30, 60, 120, 240, 480, 960, 1920]
acc_max = 0
n_h1_best, n_h2_best = 0, 0
for n_h1 in N1:
  for n_h2 in N2:
    parameters, cost_list = model(X_train, one_hot_y_train, n_h1, n_h2, 0.01, 200)
    cache = forward_prop(X_val, parameters)
    a_pred = np.argmax(cache['a3'], 0) + 1
    acc = np.sum(a_pred == y_val.T)/y_val.shape[0]

    if(acc > acc_max):
      acc_max = acc
      n_h1_best = n_h1
      n_h2_best = n_h2

In [15]:
iterations = 200
n_h1 = n_h1_best
n_h2 = n_h2_best
learning_rate = 0.01
parameters, cost_list = model(X_train, one_hot_y_train, n_h1, n_h2, learning_rate, iterations)

In [16]:
w1 = parameters['w1']
b1 = parameters['b1']
w2 = parameters['w2']
b2 = parameters['b2']
w3 = parameters['w3']
b3 = parameters['b3']

In [17]:
print(w1.shape, w2.shape, w3.shape, b1.shape, b2.shape, b3.shape)

(1920, 7) (1920, 1920) (3, 1920) (1920, 1) (1920, 1) (3, 1)


In [18]:
cache = forward_prop(X_test, parameters)
a_pred = np.argmax(cache['a3'], 0) + 1

In [19]:
def confusion_matrix(predictions, labels):
  conf_matrix = [0,0,0,0,0,0,0,0,0]
  m = labels.shape[0]
  for i in range(m):
    conf_matrix[3*(labels[i][0]-1) + predictions[i] - 1] = conf_matrix[3*(labels[i][0]-1) + predictions[i] - 1] + 1 
  return conf_matrix

In [20]:
conf_matrix = confusion_matrix(a_pred.astype(int),y_test.astype(int))

In [21]:
ind_acc1=conf_matrix[0]/sum(conf_matrix[0:3])
ind_acc2=conf_matrix[4]/sum(conf_matrix[3:6])
ind_acc3=conf_matrix[8]/sum(conf_matrix[6:9]) 
overall=(conf_matrix[0] + conf_matrix[4] + conf_matrix[8])/sum(conf_matrix)

print("Individual accuracy 1: ",ind_acc1)
print("Individual accuracy 2: ",ind_acc2)
print("Individual accuracy 3: ",ind_acc3)
print("Overall accuracy : ", overall)

Individual accuracy 1:  0.75
Individual accuracy 2:  1.0
Individual accuracy 3:  1.0
Overall accuracy :  0.9285714285714286
